In [8]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import contractions
from tqdm import tqdm

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

# Read the JSON Lines file
with open('../News_Category_Dataset_v3.json', 'r') as infile:
    lines = infile.readlines()

# Add commas between JSON objects and format as a list
with open('output.json', 'w') as outfile:
    outfile.write('[\n')  # Start of JSON array
    for i, line in enumerate(lines):
        # Strip newline characters and trailing whitespace
        line = line.strip()
        # Add comma at the end of each line except the last one
        if i < len(lines) - 1:
            outfile.write(line + ',\n')
        else:
            outfile.write(line + '\n')
    outfile.write(']\n')  # End of JSON array


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\baljyot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baljyot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\baljyot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\baljyot\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\baljyot\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\baljyot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already

In [2]:
file_name="debjit_mod.csv"
df = pd.read_csv(file_name)

In [3]:
# Count the number of NaN values in the 'article_text' column
nan_count = df['article_text'].isna().sum()
print(f"Number of NaNs in 'article_text': {nan_count}")



Number of NaNs in 'article_text': 3073


In [4]:
df.head()

,link,headline,category,short_description,authors,date,Index_Column,article_text
0,https://www.huffingtonpost.com/entry/marni-for...,Marni Mayhem: The Indignity of Shopping Fast F...,STYLE & BEAUTY,I am often mortified (post acquisition) at wha...,"Amy Tara Koch, Contributor\nAuthor, travel wri...",2012-03-12,0,"Author, travel writer and mom, lover of la dol..."
1,https://www.huffingtonpost.com/entry/star-wars...,These Incredible 3D Models Of Star Wars Land A...,ENTERTAINMENT,Disney has unveiled a detailed look at its new...,Matthew Jacobs,2017-07-14,1,"Senior Entertainment Reporter, HuffPost Here i..."
2,https://www.huffingtonpost.com/entry/family-re...,5 Keys to Cutting Your Mental and Emotional Um...,WELLNESS,It is essential to recognize that these family...,"Judith Johnson, Contributor\nI am an author, m...",2012-05-24,2,"I am an author, mentor and speaker who helps o..."
3,https://www.huffingtonpost.com/entry/how-to-ma...,Cooking For Kitchenphobes: How To Make Vinaigr...,FOOD & DRINK,Throw out those bottled dressings. They're no ...,NaN,2013-02-28,3,Cooking For Kitchenphobes is a series aimed to...
4,https://www.huffingtonpost.com/entry/five-thin...,5 Things You Should Do Every Morning,WELLNESS,You can try these simple steps to keep your mo...,"Andrea Metcalf, Contributor\nHealthy Living, C...",2012-02-08,4,"Healthy Living, Certified Trainer, Author, Spe..."


In [5]:
# Function to remove the punctuation marks from the given text
def remove_punctuation(text):

    # Returning the cleaned text for further processing
    return re.sub(r'[^\w\s]', '', text)

# Function to remove the stopwords from the given text
def remove_stopwords(text):

    # Defining a set of English stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenizing the given text
    tokens = word_tokenize(text)

    # Removing the stop words from the given text based on the above created English stopword set
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Combining tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)

    # Returning the cleaned text for further processing
    return cleaned_text

# Function to lemmatize the given text
def lemmatize_text(text):

    # Initializing the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Tokenizing the given text
    tokens = word_tokenize(text)
    
    # Lemmatizing each token of the given text to break them down to their root forms
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Combining the lemmatized tokens back into a single string abd returning it for further processing
    return ' '.join(lemmatized_tokens)

# Function to remove URLs from the given text
def remove_urls(text):

    # Defining the URL pattern to remove from the given text
    url_pattern = r'(?:(?:http|https|ftp|ftps|sftp|file|mailto|tel|ws|wss)://|www\.)[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+[^\s]*'

    # Returning the new text with all URLs removed for further processing
    return re.sub(url_pattern, '', text)

# Function to remove the html tags (if any) from the given text
def remove_html_tags(text):

    # Creating a BeautifulSoup object and parsing the text with html.parser
    soup = BeautifulSoup(text, "html.parser")

    # Getting the text without HTML tags and returning it for further processing
    return soup.get_text()

# Function to expand contractions (for example: "don't" to "do not")
def expand_contractions(text):

    # Returning the new text with expanded words for further processing
    return contractions.fix(text)

# Function to remove the numbers from the given text
def remove_numbers(text):
    
    # Returning the new text with numbers removed for further processing
    return re.sub(r'\d+', '', text)

# Function to keep only valid English words in the given text
def remove_invalid_words(text):

    # Tokenizing the given text
    tokens = word_tokenize(text)

    # # Get the list of valid English words
    word_list = set(words.words())

    # Checking validity for each word
    valid_words = [word for word in tokens if word in word_list]

    # Returning the list of valid English words extracted from the given text for further processing
    return ' '.join(valid_words)

#  Preprocessing function for cleaning the article's text for further processing
def preprocess_text(text, to_remove_stopwords = True, lemmatize = True):

    # Converting all the article's text to lowercase
    text = text.lower()

    # Removing all the urls from the article's text (keeping this before removing punctuation to avoid problems)
    text = remove_urls(text)

    # Removing all the punctuation marks from the article's text
    text = remove_punctuation(text)

    text = remove_html_tags(text) # Removing the html tags (if any)
    text = expand_contractions(text) # Expanding contractions (For example: "don't" to "do not")
    text = remove_numbers(text) # Removing numbers from the text

    text = remove_invalid_words(text)

    # Removing stop words (optional: you can keep this to improve the model's performance)
    if(to_remove_stopwords):
        text = remove_stopwords(text)
    
    # Lemmatizing the words to break them to their root meanings
    if(lemmatize):
        text = lemmatize_text(text)    

    # Returning the final clean text
    return text





# Test Example for the article text preprocessing function
text = "Hi! My name is abc. What is your name? my number is 123, https://www.youtube.com/ dlfjk kdkd dkdkdkd  <break> don't hadn't won't  "
text2 = "using https://www.google.com/ as an example"

ptext = preprocess_text(text)
ptext2 = preprocess_text(text2)
print(ptext)
print(ptext2)

# Main implementation for the rows of the article dataframe
# df['cleaned_text'] = df['article_text'].progress_apply(preprocess_text)

hi name name number break
example


In [ ]:
from tqdm import tqdm
import pandas as pd

# Enable tqdm for pandas
tqdm.pandas()

# Fill NaN values in 'article_text'
# df['article_text'] = df['article_text'].fillna('')
df['final'] = df['final'].fillna('')

# Apply the preprocess_text function with a progress bar
df['cleaned_final'] = df['cleaned_article_text'].progress_apply(lambda x: preprocess_text(x))




# Display the DataFrame with the new 'cleaned_article_text' column
# print(df[['article_text', 'cleaned_article_text']].head())


In [94]:
# Define the number of topics for NMF
n_topics = 15

# Initialize NMF model
nmf_model = NMF(n_components=15, random_state=42)

# Fit the NMF model to the combined matrix
nmf_features = nmf_model.fit_transform(doc_term_matrix)

# Create a DataFrame for the NMF features
nmf_features_df = pd.DataFrame(nmf_features, columns=[f'Component {i}' for i in range(n_topics)])

# Adding the original article IDs to the NMF features dataframe
# nmf_features_df['id'] = bm25_features_df['id']
print(nmf_features_df)

      Component 0  Component 1  Component 2  Component 3  Component 4  \
0        0.198563     0.036805     0.015591     0.176804     0.000000   
1        0.000916     0.554903     0.000000     0.202001     0.000000   
2        0.058780     0.055621     0.000000     0.179734     0.000000   
3        0.000000     0.000000     0.000000     0.200802     0.010664   
4        0.000000     0.035482     0.004741     0.111430     0.000000   
...           ...          ...          ...          ...          ...   
9995     0.000000     0.000000     0.000000     0.000000     0.000000   
9996     0.000000     0.000000     0.000000     0.000000     0.000000   
9997     0.000000     0.000000     0.000000     0.000000     0.000000   
9998     0.000000     0.000000     0.000000     0.000000     0.000000   
9999     0.000000     0.000000     0.000000     0.000000     0.000000   

      Component 5  Component 6  Component 7  Component 8  Component 9  \
0        0.000000     0.046612     0.000000     0.

In [93]:
remove_stopwords("I like to play football in the park.")

'I like play football park .'